# Load Packages

What we can recall from critisicm regarding this presentation:

- did not take the equal weighted as benchmark
- use total return indices (I think)

In [1]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

# Seaborn
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

# little function for later
def isNaN(string):
    return string != string

# Load Data

In [2]:
# rawdata is direct import from excel

# Import of excel data, sheet by sheet
xls_ETF = pd.ExcelFile("1_Data/Data Clean TM/Data_ETFxIndex_stiched.xlsx")
xls_Inflation = pd.ExcelFile("1_Data/Data Clean TM/Data_Inflation.xlsx")
xls_Yield = pd.ExcelFile("1_Data/Data Clean TM/Yields_Clean_TM.xlsx")
xls_FX = pd.ExcelFile("1_Data/Data Clean TM/Data_FX_spot.xlsx")

ETF_raw = pd.DataFrame(pd.read_excel(xls_ETF))
Inflation_raw = pd.DataFrame(pd.read_excel(xls_Inflation, 2))
Yield_long_raw = pd.DataFrame(pd.read_excel(xls_Yield, 2))
Yield_short_raw = pd.DataFrame(pd.read_excel(xls_Yield, 3))
FX_raw = pd.DataFrame(pd.read_excel(xls_FX, 3))

# keep a safe copy of the rawdata to compare the changes
ETF = ETF_raw.copy()
Inflation = Inflation_raw.copy()
Yield_long = Yield_long_raw.copy()
Yield_short = Yield_short_raw.copy()
FX = FX_raw.copy()

# Set date as index
ETF.set_index("Datum", inplace=True)
Inflation.set_index("Datum", inplace=True)
Yield_long.set_index("Datum", inplace=True)
Yield_short.set_index("Datum", inplace=True)
FX.set_index("Dates", inplace=True)

In [3]:
# Drop all the columns in which only text is and safe them separately --> ONLY RUN ONCE
# ETF_text = ETF.iloc[0,:]
# Yield_text = Yield.iloc[0,:]
# Yield_short_text = Yield_short.iloc[0,:]
# cpicore_text = cpicore.iloc[0:1,:]
# ETF.drop([0], inplace = True)
# Yield.drop([0], inplace = True)
# Yield_short.drop([0], inplace = True)
# cpicore.drop([0,1], inplace = True)

# Value

Recent infl – the average of the headline and core annual CPI inflation rate
--> we don't have headline inflation


Effective target infl - The effective inflation target is the mean of the target range announced or implied by the authorities plus an adjusted for past “target misses”, which is the last 3 years’ average gap between actual inflation and the target means
--> we don't have target rates for all countries --> use 2.5 as target rate

Formula: (1/n)*recent infl + ((n-1)/n)*effective target infl

In [4]:
# first we will take the difference between the effective target (mean) inflation and the actual inflation

cpiheadline = Inflation.copy()

eff_target_diff = pd.DataFrame(columns = cpiheadline.columns, index = cpiheadline.index)

for i in eff_target_diff.columns:
    if i == "Australia":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.025
    elif i == "Canada":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "China":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Germany":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "France":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "UK":
        eff_target_diff.loc[:"2003-11-28",i] = cpiheadline.loc[:"2003-11-28",i] - 0.025
        eff_target_diff.loc["2003-12-31":,i] = cpiheadline.loc["2003-12-31":,i] - 0.02
    elif i == "Italy":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Japan":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.033
    elif i == "Singapore":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "USA":
        eff_target_diff.loc[:"2011-12-31",i] = cpiheadline.loc[:"2011-12-31",i] - 0.0185
        eff_target_diff.loc["2012-01-31":,i] = cpiheadline.loc["2012-01-31":,i] - 0.02
    elif i == "Spain":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.02
    elif i == "Switzerland":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.01
    elif i == "India":
        eff_target_diff.loc[:,i] = cpiheadline.loc[:,i] - 0.04

In [5]:
np.nanmean(eff_target_diff.loc[:"2002-11-30", "Singapore"])
#np.nanmean(eff_target_diff.loc["2020-04-30":, "Singapore"])

-0.012722222222222225

In [6]:
# create the rolling 3-year average of the core inflation

# first I have to set up an empty dictionary to store the rolling averages
cpiheadline_avg = {}
for i in cpiheadline.columns:
    cpiheadline_avg[i] = []

# next up we iterate over the cpicore data to get the index and safe them in the dictionary
runner = len(cpiheadline.index)
for i in cpiheadline.index:
    if runner == 35:
        break
    
    for j,k in enumerate(cpiheadline.columns):
        inflation = np.nanmean(eff_target_diff.iloc[runner-36:runner, j])
        cpiheadline_avg[k].insert(0, inflation)
    runner -= 1
    
#cpiheadline_avg
cpiheadline_avg_trimmed = pd.DataFrame(cpiheadline_avg)

AttributeError: 'float' object has no attribute 'dtype'

In [ ]:
cpiheadline_avg_trimmed # until here I checked several times whether it is correct and it seems to work

In [ ]:
# now we calculate the value formula:
# (1/n)*recent inflation + ((n-1)/n)*effective target inflation for the years: 2, 5, 10, 20, 30

# first we need to align the dataframes 
cpiheadline_trimmed = cpiheadline[35:]
cpiheadline_avg_trimmed.set_index(cpiheadline_trimmed.index, inplace = True)

# second we create an empty dictionary to fill with the calculations
formula_values = {}
x = 0
for i in cpiheadline_trimmed.columns:
    x += 1
    for j in [2,5,10,20,30]:
        formula_values[f"{i},{j}"] = []
formula_values = pd.DataFrame(columns = formula_values.keys(), index = cpiheadline_trimmed.index)


# now that we have the empty dictionary to fill, we start with caluclating the inflation expectation
for i,k in enumerate (cpiheadline_trimmed.columns):
    for j,m in enumerate([2,5,10,20,30]):
        formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]
        
#formula_values

<ipython-input-5-90c02453a451>:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  formula_values.iloc[:,j+i*5] = (1/m)*cpiheadline_trimmed[k] +((m-1)/m)*cpiheadline_avg_trimmed[k]


In [ ]:
# now we need to align the NA's from the ETF with the NA's from the Yields such that we don't get a weight for a date that
# we don't have return data for

Yield_na = Yield_long.copy()
ETF_na = ETF.pct_change().isna()
ETF_na
runner = 0
counter = 0
for i,k in enumerate(Yield_na.columns):
    if counter == 5:
        counter = 0
        runner += 1
    tmp = ETF_na.iloc[:,runner]
    Yield_na.iloc[tmp,i] = np.nan
    counter +=1
Yield_na


# now we have the issue that the Yield_clean data and the Inflation_clean data doesn't have the same sequence of countries
# to avoid this we resort the Yield_clean data to fit the Inflation_clean data

clean_names = ["Germany", "France","Spain","Italy","United Kingdom", "Switzerland", "China", "Australia", "Japan", "USA", "Canada", "Singapore", "India"]

# we split the dataframe into the different countries...
Germany = Yield_na.iloc[:,:5]
France = Yield_na.iloc[:,5:10]
Spain = Yield_na.iloc[:,10:15]
Italy = Yield_na.iloc[:,15:20]
UK = Yield_na.iloc[:,20:25]
Switzerland = Yield_na.iloc[:,25:30]
China = Yield_na.iloc[:,30:35]
Australia = Yield_na.iloc[:,35:40]
Japan = Yield_na.iloc[:,40:45]
USA = Yield_na.iloc[:,45:50]
Canada = Yield_na.iloc[:,50:55]
Singapore = Yield_na.iloc[:,55:60]
India = Yield_na.iloc[:,60:]

# ... and fusion them together to have the same order as the Inflation data (plus dividing by 100 to have same level as Inflation)
Yields_aligned = [India, Singapore, Canada, USA, Japan, Australia, China, Switzerland, UK, Spain, France, Germany, Italy]
Yields_aligned = pd.concat(Yields_aligned, axis=1)
Yields_aligned = Yields_aligned /100
#Yields_aligned

In [ ]:
# in a next step we adjust the yield data to have the same lookback as the other data

Yield_trimmed = Yields_aligned[35:]

Real_yield = pd.DataFrame(0, columns = formula_values.columns, index = formula_values.index)

# now we take the difference between yield etf and inflation
for i,k in enumerate(formula_values.columns):
    for j,l in enumerate(formula_values.index):
        Real_yield.iloc[j,i] = Yield_trimmed.iloc[j,i] - formula_values.iloc[j,i]
#Real_yield

In [ ]:
# in a final step we will average over the duration of the countries to get one average for each country

Real_yield_avg = pd.DataFrame(columns = cpiheadline_avg_trimmed.columns, index = Real_yield.index)

runner = 0
for i in range(0, len(Yields_aligned.columns)-1, 5):
    temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
    Real_yield_avg.iloc[:,runner] = temp.values
    runner +=1
    
#Real_yield_avg

<ipython-input-8-1ebfb3f6f625>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))
<ipython-input-8-1ebfb3f6f625>:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Real_yield_avg.iloc[:,runner] = temp.values
<ipython-input-8-1ebfb3f6f625>:7: RuntimeWarning: Mean of empty slice
  temp = pd.DataFrame(np.nanmean([Real_yield.iloc[:,i],Real_yield.iloc[:,i+1],Real_yield.iloc[:,i+2],Real_yield.iloc[:,i+3],Real_yield.iloc[:,i+4]], axis = 0))


# Momentum

In [42]:
def computeTurnover(previousWeights, newWeights, assetReturns, Rf):
    """
    Computes turnover by comparing the previous and the new target weights 
    of the portfolio, accounting for the returns on the assets. The function 
    also computes the portfolio return excluding transaction costs, Rp. 

    Parameters:
        previousWeights (array-like): previous target weights of the portfolio
        newWeights (array-like): new target weights of the portfolio
        assetReturns (array-like): returns on the assets
        Rf (float): risk-free rate of return

    Returns:
        turnover (float): turnover
        Rp (float): portfolio return excluding transaction costs
    """

    Rp = np.dot(previousWeights, assetReturns) + (1 - np.sum(previousWeights)) * Rf
    valuePerAsset = previousWeights * (1 + assetReturns)
    currentWeights = valuePerAsset / (1 + Rp)
    turnover = np.sum(np.abs(newWeights - currentWeights))

    return turnover, Rp

In [50]:
# Create Monthly Returns out of ETF Prices
ETF_price = ETF.copy()
ETF_returns = ETF_price.pct_change()
ETF_ret = ETF_returns.tail(-1)
rf = Yield_short.iloc[:,0] / 100

In [51]:
# Create Weights for Momentum

# Prep Work
nAssets = len(ETF_price.columns) # 13 Countries
nMonths = len(ETF_price) # 280 Months of prices (31.12.1999 - 31.03.2023)
lookbackStart = 12
lookbackEnd = 1
firstMonth = lookbackStart + 1 # we can only start computing weights in 13.month bc we have 12 months lookback 
nLongs = 5
nShorts = 5
momLongWeights = pd.DataFrame(np.zeros((nMonths, nAssets)))
momLSWeights = pd.DataFrame(np.zeros((nMonths, nAssets)))
long_weight = np.divide(1,nLongs)
short_weight = np.divide(1,nShorts)*(-1)

In [52]:
 # Loop to get the Long weights and the Long/Short Weights

for month in range(firstMonth, nMonths):
     pastReturns = pd.DataFrame(np.divide(ETF_price.iloc[month - lookbackEnd, :], ETF_price.iloc[month - lookbackStart, :]) - 1)
     highReturns = pastReturns.nlargest(nLongs,columns=pastReturns.columns)
     lowReturns = pastReturns.nsmallest(nShorts,columns=pastReturns.columns)
     low_index = lowReturns.index
     high_index = highReturns.index
     pRet_index = pastReturns.index

     high_index_numbers = pRet_index.get_indexer(high_index)
     low_index_numbers = pRet_index.get_indexer(low_index)
     # Long Weights only
     momLongWeights.iloc[month,high_index_numbers] = long_weight
     # Long/Short Weights only
     momLSWeights.iloc[month,high_index_numbers] = long_weight
     momLSWeights.iloc[month,low_index_numbers] = short_weight

# Set Index and Column Names again
momLongWeights = momLongWeights.set_index(ETF_price.index).copy()
momLongWeights.columns = ETF_price.columns
momLSWeights = momLSWeights.set_index(ETF_price.index).copy()
momLSWeights.columns = ETF_price.columns


In [53]:
# Data Cleanup -> Returns in sync with weights!
momLongWeights_cut = momLongWeights.iloc[firstMonth:, :] # cut out first 13 months -> first row now is weight calculated at 31.01.2000; keep extra row at end tor Turnover
momLSWeights_cut = momLSWeights.iloc[firstMonth:, :] # cut out first 13 months -> first row now is weight calculated at 31.01.2000; keep extra row at end tor Turnover
ETF_ret_cut = ETF_ret.iloc[firstMonth :, :] # Not firstMonth+1 bc we dropped one row above in ret calculation -> we need Returns from 28.02.2000
rf_cut_mom = rf.iloc[firstMonth+1:]
dates4Fig = ETF_ret_cut.index
nMonths_actual = nMonths - (firstMonth+1)

In [54]:
# Calculate strategy Returns no TC
stratReturnsNoTC = pd.DataFrame(np.zeros((nMonths_actual, 2)))
# note we only take weights rows until end -1 bc we left one to much in
LongStratRet = np.sum(np.multiply(ETF_ret_cut, momLongWeights_cut.iloc[ : -1, :]), axis=1)
LSStratRet = np.sum(np.multiply(ETF_ret_cut, momLongWeights_cut.iloc[ : -1, :]), axis=1) + rf_cut_mom

/var/folders/p7/g7d04mzx5wn88jp9x3dzpb9w0000gn/T/ipykernel_37515/2365980571.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  LongStratRet = np.sum(np.multiply(ETF_ret_cut, momLongWeights_cut.iloc[ : -1, :]), axis=1)
/var/folders/p7/g7d04mzx5wn88jp9x3dzpb9w0000gn/T/ipykernel_37515/2365980571.py:5: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Seri

In [55]:
LSStratRet

Datum
2001-02-28    0.052459
2001-03-30    0.051580
2001-04-30    0.038279
2001-05-31    0.046413
2001-06-29    0.050453
                ...   
2022-11-30    0.026126
2022-12-30    0.012446
2023-01-31    0.026195
2023-02-28    0.005653
2023-03-31    0.040373
Length: 266, dtype: float64

In [27]:
pd.DataFrame([LongStratRet,LSStratRet]) #Not done yet

Datum,1999-12-31,2000-01-31,2000-02-29,2000-03-31,2000-04-28,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-29,...,2022-06-30,2022-07-29,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-30,2023-01-31,2023-02-28,2023-03-31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.012201,0.014272,-0.010344,-0.025153,-0.004216,0.010806,-0.002874,0.010875,-0.009667,0.025053
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.218201,0.001272,0.293656,1.136847,1.527784,1.542806,1.529126,1.542875,1.522333,1.557053


In [56]:
pd.concat([LongStratRet,LSStratRet],axis=1,co)

,0,1
Datum,,
2001-02-28,0.004859,0.052459
2001-03-30,0.005880,0.051580
2001-04-30,-0.010121,0.038279
2001-05-31,0.001013,0.046413
2001-06-29,0.005453,0.050453
...,...,...
2022-11-30,0.010806,0.026126
2022-12-30,-0.002874,0.012446
2023-01-31,0.010875,0.026195


# Carry

In [ ]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M
# I take the Yield_na for the calculation to get the same NA's as the ETF has

Carry_3m = pd.DataFrame(columns = Yield_long.columns, index = Yield_long.index)
avg_Carry_3m = pd.DataFrame(0, columns = Yield_short.columns, index = Yield_short.index)

runner = 0
counter = 0
for i in range(0,len(Yield_long.columns)):
    if counter == 5:
        runner += 1
        counter = 0
    Carry_3m.iloc[:,i] = Yield_na.iloc[:,i] - Yield_short.iloc[:,runner]
    counter += 1
    
for i in range(0,len(Yield_short.columns)):
    avg_Carry_3m.iloc[:,i] = np.nanmean([Carry_3m.iloc[:,i*5], Carry_3m.iloc[:,i*5+1], Carry_3m.iloc[:,i*5+2], Carry_3m.iloc[:,i*5+3], Carry_3m.iloc[:,i*5+4]], axis = 0)

avg_Carry_3m = avg_Carry_3m / 100

<ipython-input-9-fb6f15460685>:14: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  Carry_3m.iloc[:,i] = Yield_na.iloc[:,i] - Yield_short.iloc[:,runner]
<ipython-input-9-fb6f15460685>:18: RuntimeWarning: Mean of empty slice
  avg_Carry_3m.iloc[:,i] = np.nanmean([Carry_3m.iloc[:,i*5], Carry_3m.iloc[:,i*5+1], Carry_3m.iloc[:,i*5+2], Carry_3m.iloc[:,i*5+3], Carry_3m.iloc[:,i*5+4]], axis = 0)


# Sort function to create the buckets

In [ ]:
def pmp_Sort(sort_variable, n_longs, n_shorts, long_high_values=1, equal_weights_ls=False):
    """
    Generates portfolio weights based on sort_variable. The function ignores assets for which the sort variable is
    missing (NaN). All such assets get a weight of zero in the portfolio. n_longs and n_shorts denote the number of
    assets held long and short. When long_high_values is True, assets that have the highest values for sort_variable
    are held long and those with the lowest values are held short. Otherwise, the opposite holds.
    """
    # Find the assets with the highest and lowest values of the sort variable
    if long_high_values:
        sort_variable_sorted = sort_variable  # For long high values, sort descending
    else:
        sort_variable_sorted = -sort_variable   # For long low values, sort ascending
    non_nan_assets = ~np.isnan(sort_variable_sorted)
    non_nan_count = np.count_nonzero(non_nan_assets)
    
    # Check if n_shorts is zero
    if n_shorts == 0:
        tmp_long = list(sort_variable_sorted)
        tmp_long = [-10000000 if pd.isna(x) else x for x in tmp_long]
        list_of_longs = np.argsort(tmp_long)[-5:]
        
        weights = np.zeros(len(sort_variable))
        weights[list_of_longs] = 1 / n_longs
        return weights

    # If n_shorts is not zero, handle normally
    if non_nan_count < n_longs:
        n_longs = non_nan_count
        n_shorts = 0
    elif n_shorts > 0 and non_nan_count < n_longs + n_shorts:
        n_shorts = non_nan_count - n_longs
        
    if equal_weights_ls == True:
        n_shorts = non_nan_count // 2
        n_longs = non_nan_count // 2
        
    

    tmp_long = list(sort_variable_sorted)
    tmp_long = [-10000000 if pd.isna(x) else x for x in tmp_long]
    list_of_longs = np.argsort(tmp_long)[-5:]
        
    tmp_short = list(sort_variable_sorted)
    tmp_short = [100000000 if pd.isna(x) else x for x in tmp_short]
    list_of_shorts = np.argsort(tmp_short)[:5]
    

    # Assign the weights to assets in the list of longs and shorts
    n_assets = len(sort_variable)
    weights = np.zeros(n_assets)
    weights[list_of_longs] = 1 / n_longs
    weights[list_of_shorts] = -1 / n_shorts

    return weights


In [ ]:
# Value
weights = pd.DataFrame(columns = Real_yield_avg.columns, index = Real_yield_avg.index)

for i,k in enumerate(Real_yield_avg.index):
    temp = pmp_Sort(Real_yield_avg.iloc[i,:], 5,5)
    weights.iloc[i,:] = temp
    
# now we need to align the columns of weights to columns of returns
ETF_returns = ETF.pct_change()
ETF_value = ETF_returns.iloc[35:,:]
ETF_value.rename(columns = {"Swiss comb": "Switzerland"}, inplace=True)

Value_weights = pd.DataFrame(columns = ETF_value.columns, index = ETF_value.index)
weights.set_index(ETF_value.index, inplace=True)
for i in ETF_value.columns:
    Value_weights.loc[:,i] = weights.loc[:,i]

#Value_weights

<ipython-input-27-59ec5279197b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ETF_value.rename(columns = {"Swiss comb": "Switzerland"}, inplace=True)


In [ ]:
# Momentum

In [ ]:
# Carry
Carry_weights = pd.DataFrame(columns = avg_Carry_3m.columns, index = avg_Carry_3m.index)

for i,k in enumerate(avg_Carry_3m.index):
    if i == 0:
        continue
    temp = pmp_Sort(avg_Carry_3m.iloc[i,:], 5, 5)
    Carry_weights.iloc[i,:] = temp

#Carry_weights

# PF weights (not yet working)

In [ ]:
# in the following we will combine the weight matrices that we've gathered sofar into two distinct weight matrices for the PF
# the PF_weight_score will be a number between -3 and 3 indicating the signals suggesting the country
# the PF_weight will just be the sum of the weights

PF_weights = Value_weights + Carry_weights

# now we will create the scored weights
PF_score = pd.DataFrame(columns = Value_weights.columns, index = Value_weights.index)
PF_weights_score = pd.DataFrame(columns = Value_weights.columns, index = Value_weights.index)

value_pos = Value_weights > 0
value_neg = Value_weights < 0
carry_pos = Carry_weights > 0
carry_neg = Carry_weights < 0

PF_score[value_pos] = 1
PF_score[value_neg] = -1
PF_score[carry_pos] += 1
PF_score[carry_neg] -= 1

# print(PF_weights)
# print(PF_weights_score)

# now we will create the weights for the PF_weights_score
x = PF_score > 0
y = PF_score < 0
PF_pos = PF_score[x]
PF_neg = PF_score[y]

for i,k in enumerate(PF_pos.index):
    pos = sum(PF_pos.iloc[i,:]>0)
    if pos == 0:
        continue
    pos = 1/pos
    
    neg = sum(PF_score.iloc[i,:]<0)
    if neg == 0:
        continue
    neg = -1/neg
    
    PF_weights_score.iloc[i,:] = PF_pos.iloc[i,:] * pos
    PF_weights_score.iloc[i,:] = PF_neg.iloc[i,:] * neg
    
    

In [ ]:
PF_weights_score

,Germany,France,Spain,Italy,UK,Switzerland,China,Australia,Japan,USA,Canada,Singapore,India
Datum,,,,,,,,,,,,,
2002-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,0.666667,NaN,NaN,NaN,0.333333,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-30,0.666667,NaN,NaN,NaN,0.333333,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31,0.666667,NaN,NaN,NaN,0.333333,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Return calculation

In [ ]:
# now that we have the weights of the different strategies and the returns of the ETF's we can calculate the returns

# Value

# first we need to align the length of the dataframes and the columns of the two dataframes to match each other
Value_returns = Value_weights.mul(ETF_value)
#Value_returns

# Value_returns_2 = pd.DataFrame(columns = Value_weights.columns, index = ETF_value.index)
# for i,k in enumerate(Value_weights.columns):
#     Value_returns_2.iloc[:,i] = ETF_value.iloc[:,i] * Value_weights.iloc[:,i]
# Value_returns_2.tail(10)

In [ ]:
Carry_returns = Carry_weights.mul(ETF_returns)
#Carry_returns

# Transaction cost

In [ ]:
# we will be using the ETF_returns, and the respective weights for the calcuclation of the transaction costs
# furthermore we need the return of the whole portfolio to calculate the TC

# Converting returns to Euro

# Benchmarking

In [ ]:
benchmark = df[["MSCI_EU"]]
benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
benchmark

In [ ]:
bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
bm_mtl_ret


In [ ]:
#compare momentum result vs benchmark
comparison = pd.DataFrame(bm_mtl_ret)
comparison["mom_ret"] = profits
comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
comparison


In [ ]:
comparison

# Comparison Plot

In [ ]:
# Plotting
fig, ax = plt.subplots()
ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
ax.legend(loc="best")
ax.set_ylabel('Monthly Returns')
ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")